# LabOP Labware development

Why using a EMMOntoPy base approach

- the ontology is formulated programmatically in python
- supports developer with a lot of automatic functionality, like using UUIDs as IDs (by default), SKOS-labels (prefered label) , annotations, ...
- includes dimension and unit support out of the box (no extra definition required) - also many other concepts that are useful 
- unifies the way, the OWL based ontology is generated
- has verification tool to check the syntactic consistency of the ontology
- fast FACT++ reasoner for logic consistency check
- easy generation of turtle (ttl) OWL files and many other formats
- basic query is directly supported
- integrated SPRAQL engine for advance queries

 - everything is encapsulated in python "classes", that can directly be used in applications (!!)


## Design guidelines for the labware ontology

- as general as possible -> wide applicability
- capturing the most important/common labware features
- clear separation of Terminology part (T-Box, abstract class definitions) and Assertion Part (A-Box, individuals)

 ## outlook

 - improved properties / relations (proper auto - unit support)
 - packing everything into a nice package
 - copying code to LapOP repository (best in a new project)

## open questions

* tolerances ?
-> unit conversion : owl / ttl file -> Robert
* shape representation ? STL file ? .AMF format, is the modern alternative to .STL format 
* first interaction position (e.g. for a tube rack, which position is the first one) ? Vector ?
* Width vs. diameter (e.g. for a tube, which one is the diameter ?)
* granularity of the materials (e.g. for a tube rack, which material is the rack made of) - reference to other ontologies?
* how to handle the "has part" relation (e.g. for a tube rack, which has tubes as parts)
* how to handle versioning of the ontology ?
* color representation (e.g. for a tube rack, which color is the rack) - RGB, HEX, CMYK, ... ?
* !!! disjoint
* reusable 
* transparency

- further ideas:
- surface
- adhesion (mid-binding, high-binding, ...)
- construction style - single piece, welded, ...
- liquid contact yes/no

Scoping:
Must-have: SLAS (“SBS”)
Likely should: flasks, tubes, tube racks, tips and tip boxes
“stuff that lives on a lot of people’s robot decks”
—-------------- (the line of inclusion) —-------------
Nice-to-have: larger reactors
Must not: custom blown glassware
“ 


In [ ]:
#  currently only reflib 4.2.1 is supported
#! pip install rdflib==4.2.1
#! pip install packaging==21.0
import os
import pathlib
import logging
import pandas as pd

from ontopy import World
from ontopy.utils import write_catalog

import owlready2
from owlready2 import DatatypeProperty, FunctionalProperty, ObjectProperty, AllDisjoint

# --- helper functions

def en(s):
    """Returns `s` as an English location string."""
    return owlready2.locstr(s, lang='en')


def pl(s):
    """Returns `s` as a plain literal string."""
    return owlready2.locstr(s, lang='')

## Labware Ontology Terminology Component (TBox) 

In [ ]:
class labop_Labware:
    def __init__(self, emmo_world=None) -> None:

        __version__ = "0.0.1"
        __file__ = "."

        self.__version__ = __version__

        self.labop_labware_base_iri = 'http://www.oso.org/oso/labware#'
        self.labop_labware_version_iri = f'http://www.oso.org/{__version__}/oso/labware'

        output_filename_base = os.path.join('labop_labware_base')
        self.labop_labware_owl_filename = f'{output_filename_base}-v{__version__}.owl'
        self.labop_labware_ttl_filename = f'{output_filename_base}-v{__version__}.ttl'

        # EMMO top-level ontology 

        self.emmo_url = (
            'https://raw.githubusercontent.com/emmo-repo/emmo-repo.github.io/'
            'master/versions/1.0.0-beta/emmo-inferred-chemistry2.ttl')

        # assign a local path fo the ontology, in case one wants to work offline

        self.emmo_url_local = os.path.join(pathlib.Path(
            __file__).parent.resolve(), "emmo", "emmo-inferred-chemistry2")

        if os.path.isfile(self.emmo_url_local + '.ttl'):
            self.emmo_url = self.emmo_url_local

        #self.emmo_world = World(filename="emmo_labwares.sqlite3") # this could manifest the ontology in a database
        if emmo_world is not None:
            self.emmo_world = emmo_world
        else:
            self.emmo_world = World()
            
            # loading the EMMO top-level ontology
            self.emmo = self.emmo_world.get_ontology(self.emmo_url)
            self.emmo.load()  # reload_if_newer = True
            self.emmo.sync_python_names()  # Synchronise annotations
            self.emmo.base_iri = self.emmo.base_iri.rstrip('/#')
            self.catalog_mappings = {self.emmo.base_iri: self.emmo_url}

        # Create new ontology: labOP-labware - lolw
        self.lolw = self.emmo_world.get_ontology(self.labop_labware_base_iri)
        if emmo_world is None:
            self.lolw.imported_ontologies.append(self.emmo)
        self.lolw.sync_python_names()

    # defining the  labOP-labware ontology
    def define_ontology(self):
        logging.debug('defining labware ontology')

        with self.lolw:

            # Terminology Component (TBox) 

            # Basic Relations
            # ================

            class hasType(self.lolw.hasConvention):
                """Associates a type (string, number...) to a property."""

            class isTypeOf(self.lolw.hasConvention):
                """Associates a property to a type (string, number...)."""
                inverse_property = hasType

            # Physical Properties
            # ====================

            class Length(self.emmo.Length):
                """"Length
                extends EMMO:Length
                """
                wikipediaEntry = en("https://en.wikipedia.org/wiki/Length")

                # add reference SI unit
                referenceUnit = self.emmo.Metre

                length = 0.0

            class Volume(self.emmo.Volume):
                """Total Labware volume """
                physicalDimension = pl("T+1 L3 M0 I0 Θ0 N0 J0")
                wikipediaEntry = en("https://en.wikipedia.org/wiki/Volume")

                # add reference SI unit
                referenceUnit = self.emmo.CubicMetre

            class Mass(self.emmo.Mass):
                """Total Labware mass """
                physicalDimension = pl("T+1 L0 M1 I0 Θ0 N0 J0")
                wikipediaEntry = en("https://en.wikipedia.org/wiki/Mass")

                # add reference SI unit
                referenceUnit = self.emmo.Kilogram

            class Force(self.emmo.Force):
                """Force of a labware, e.g. for screw caps""" 
                # add quantity“Scoping:

                physicalDimension = pl("T+1 L1 M1 I0 Θ0 N-2 J0")
                wikipediaEntry = en("https://en.wikipedia.org/wiki/Force")

                # reference SI unit
                referenceUnit = self.emmo.Newton

            class Torque(self.emmo.Torque):
                """Torque of a labware, e.g. for screw caps""" 
                # add quantity“Scoping:

                physicalDimension = pl("T+1 L2 M1 I0 Θ0 N0 J0")
                wikipediaEntry = en("https://en.wikipedia.org/wiki/Torque")


                # reference SI unit
                referenceUnit = self.emmo.NewtonMetre
                
            AllDisjoint([Torque, Length, Volume])

            class Material:
                """Polymer, properties, like solvent tolerance, transparency, ...."""
                wikipediaEntry = en("https://en.wikipedia.org/wiki/Material")


            class Model3D:
                """3D model of the labware in X format. STL ?"""
                wikipediaEntry = en("https://en.wikipedia.org/wiki/3D_modeling")

            
            class ColorRGB:
                """Labware color, e.g. RGB value"""
                wikipediaEntry = en("https://en.wikipedia.org/wiki/Color")

           
           # AllDisjoint([Material, Model3D, ColorRGB])


            # multiwell labware
            # =================

            class WellVolume(self.emmo.Volume):
                """Total Labware volume """

            class WellDistRow(self.emmo.Length):
                """wWll-to-well distance in row direction"""
            
            class WellDistCol(self.emmo.Length):
                """"Well-to-well distance in column direction"""

            #AllDisjoint([WellVolume, WellDistRow, WellDistCol])

            # Well properties of labware with wells
            class DepthWell(self.emmo.Length):
                """Well total well depth=hight"""
            
            class ShapeWell:
                """Well overall / top well shape,e.g. round, square, buffeled,..."""
            
            class ShapeWellBottom:
                """Well, bottom shape, flat, round, conical-"""

            class TopRadiusXY(self.emmo.Length):
                """Well radius of a round well at the top opening in x-y plane."""

            class BottomRadiusXY(self.emmo.Length):
                """Radius of a round bottom in xy plane / direction."""

            class BottomRadiusZ(self.emmo.Length):
                """Radius of a round bottom in z (hight) direction."""

            class ConeAngle(self.emmo.Angle):
                """Opening angle of cone in deg."""

            class ConeDepth(self.emmo.Length):
                """Depth of cone from beginning of conical shape."""

            class ShapePolygonXY:
                """Generalized shape polygon for more complex well shapes, in xy plane / direction."""

            class ShapePolygonZ:
                """Generalized shape polygon for more complex well shapes, in z direction = rotation axis."""

            class ShapeModel2D:
                """2D model of Well shape"""

            class ShapeModel3D:
                """3D model of Well shape"""

            class FirstInteractionPosition(self.emmo.Vector):
                """Position of first interaction point of a pipette tip with a well or a needle with a septum, rel. to the upper left corner of the labware. - what about round labware?"""


            #AllDisjoint([DepthWell, ShapeWell, ShapeWellBottom, TopRadiusXY, BottomRadiusXY, BottomRadiusZ, ConeAngle, ConeDepth, ShapePolygonXY, ShapePolygonZ, ShapeModel2D, ShapeModel3D, FirstInteractionPosition])

            
            # Labware Classes
            # ====================

            # Basic ------

            class Labware(self.lolw.Device):
                """Labware is a utility device that all experiments are done with and which is not actively measuring. Examples: a container, a pipette tip, a reactor, ... """
                wikipediaEntry = en("https://en.wikipedia.org/wiki/Labware")

                # is_a = [self.lolw.has_Material.some(str),
                #         self.lolw.has_NumCols.some(int),
                #         self.lolw.has_NumRows.some(int)]

            #  Relations / Properties
            # ========================

            # Physical Properties

            #class hasLength:
                # """"Labware total length """
                # is_a = [
                #     self.lolw.hasReferenceUnit.only(
                #         self.lolw.hasPhysicalDimension.only(self.lolw.Length)
                #     ),
                #     hasType.exactly(1, self.lolw.Real), ]

            # class hasWidth(FunctionalProperty):
            #     """Labware total width, """
            #     domain = [Labware]
            #     range = [Width]

            # class hasHeight(Labware >> self.lolw.Height, FunctionalProperty):
            #     """Labware total hight, without  any additions, like lids etc. """

            # class hasLengthTolerance(Labware >> self.emmo.Length, FunctionalProperty, ObjectProperty):
            #     """Labware length tolerance."""

            class hasLength(Labware >> Length, FunctionalProperty, ObjectProperty):
                """Labware total length, without  any additions, like lids etc."""
                

            class hasLengthTolerance(Length >> float, FunctionalProperty, DatatypeProperty):
                """Labware relative length tolerance (= measured width/target width)."""
            
            class hasWidth(Labware >> Length, FunctionalProperty, ObjectProperty):
                """Labware total width, without  any additions, like lids etc."""
            
            class hasWidthTolerance(Length >> float, FunctionalProperty, DatatypeProperty):
                """Labware relative width tolerance (= measured width/target width)."""
            
            class hasHeight(Labware >> Length, FunctionalProperty, ObjectProperty):
                """Labware total hight, without  any additions, like lids etc. """

            class hasHeightTolerance(Length >> float, FunctionalProperty, DatatypeProperty):
                """Labware height tolerance."""

            class hasGrippingHeight(Labware >> Length, FunctionalProperty, ObjectProperty):
                """Labware total hight, without  any additions, like lids etc. """

            class hasGrippingHeightLidding(Labware >> Length, FunctionalProperty, ObjectProperty):
                """Labware total hight, without  any additions, like lids etc. """
            
            class hasGrippingHeightWithLid(Labware >> Length, FunctionalProperty, ObjectProperty):
                """Labware total hight, without  any additions, like lids etc. """

            class hasRadiusXY(Labware >> Length, FunctionalProperty, ObjectProperty):
                """Labware radius of a round shape in XY direction """

            class hasRadiusZ(Labware >> Length, FunctionalProperty, ObjectProperty):
                """Labware radius of a round shape in XY direction """

            class hasVolume(Labware >> float, FunctionalProperty):
                """Total Labware volume """

            class hasHightLidded(Labware >> float, FunctionalProperty):
                """Labware total hight, with additions, like lids etc."""

            class hasHightStacked(Labware >> float, FunctionalProperty):
                """Labware stacking height without any additions, like lids."""

            class hasHightStackedLidded(Labware >> float, FunctionalProperty):
                """Labware stacking height with additions, like lids."""

            class hasMass(Labware >> float, FunctionalProperty):
                """Mass of the Labware """

            class hasMaxSheerForce(Labware >> self.emmo.Force, FunctionalProperty):
                """Max sheer force of the Labware, e.g. during centrifugation"""

            class hasCoatingMaterial(Labware >> str, FunctionalProperty):
                """Labware coating material"""

            
            class hasColor(Labware >> str, FunctionalProperty):
                """Labware color in RGB hex encoding"""

            class isLiddable(Labware >> bool, FunctionalProperty):
                """labware is liddable"""

            class isStackable(Labware >> bool, FunctionalProperty):
                """labware is stackable"""

            class isSealable(Labware >> bool, FunctionalProperty):
                """container is sealable"""

            class hasSetptum(Labware >> bool, FunctionalProperty):
                """Setptum of the Labware"""

            class hasMaterial(Labware >> str, DatatypeProperty):
                """Polymer, properties, like solvent tolerance, transparency, ...."""

            class hasSeptumMaterial(Labware >> str, FunctionalProperty):
                """Septum material"""

            class hasSeptumPenetrationForce(Labware >> self.emmo.Force, FunctionalProperty):
                """Septum penetration force"""


            # multiwell labware

            class hasNumCols(Labware >> int, FunctionalProperty):
                """Number of Columns of muti-well labware"""

            class hasNumRows(Labware >> int, FunctionalProperty):
                """Number of Rows of Labware"""

            class hasNumWells(Labware >> int, FunctionalProperty):
                """Number of Wells of muti-well labware"""

            # Production Properties / Metadata

            class hasManifacturer(Labware >> str, FunctionalProperty):
                 """Name of the Manufacturer """
            
            class isProductType(Labware >> str, FunctionalProperty):
                """Labware product Type"""

            class hasModelNumber(Labware >> str, FunctionalProperty):
                """Labware model number"""

            class hasProductNumber(Labware >> str, FunctionalProperty):
                """Manufacturer Product Number of the Labware"""

            class hasWellVolume(Labware >> float, FunctionalProperty):
                """Total Labware volume """

            class hasWellDistRow(Labware >> float, FunctionalProperty):
                """wWll-to-well distance in row direction"""
            
            class hasWellDistCol(Labware >> float, FunctionalProperty):
                """"Well-to-well distance in column direction"""

            # Well properties of labware with wells
            class hasDepthWell(Labware >> float, FunctionalProperty):
                """Well total well depth=hight"""
            
            class hasShapeWell(Labware >> str, FunctionalProperty):
                """Well overall / top well shape,e.g. round, square, buffeled,..."""
            
            class hasShapeWellBottom(Labware >> str, FunctionalProperty):
                """Well, bottom shape, flat, round, conical-"""

            class hasTopRadiusXY(Labware >> float, FunctionalProperty):
                """Well radius of a round well at the top opening in x-y plane."""

            class hasBottomRadiusXY(Labware >> float, FunctionalProperty):
                """Radius of a round bottom in xy plane / direction."""

            class hasBottomRadiusZ(Labware >> float, FunctionalProperty):
                """Radius of a round bottom in z (hight) direction."""

            class hasConeAngle(Labware >> float, FunctionalProperty):
                """Opening angle of cone in deg."""

            class hasConeDepth(Labware >> float, FunctionalProperty):
                """Depth of cone from beginning of conical shape."""

            class hasShapePolygonXY(Labware >> float, FunctionalProperty):
                """Generalized shape polygon for more complex well shapes, in xy plane / direction."""

            class hasShapePolygonZ(Labware >> str, FunctionalProperty):
                """Generalized shape polygon for more complex well shapes, in z direction = rotation axis."""

            class hasShapeModel2D(Labware >> str, FunctionalProperty):
                """2D model of Well shape"""

            class hasShapeModel3D(Labware >> str, FunctionalProperty):
                """3D model of Well shape"""

            # labware with screw cap

            class hasScrewCap(Labware >> bool, FunctionalProperty):
                """Screw cap type"""

            class hasScrewCapMaterial(Labware >> str, FunctionalProperty):
                """Screw cap material"""
            
            class hasScrewCapColor(Labware >> str, FunctionalProperty):
                """Screw cap color"""

            
            # further properties:

            # lengthAtEdge, lengthOverall, isSLAS1-2004complian

            # isSLAS1-2004compliant

            

            # all disjoined properties

            # special labware classes
            # can be used for faster type testing
            # ===================================================

            class SLAS_4_2004_96_Well_Plate(Labware):
                """96 Well Microtiter Plate according to SLAS 4-2004 standard"""
                equivalent_to = [ Labware & hasNumCols.value(12) &  hasNumRows.value(8) & hasNumWells.value(96) 
                                & hasWellVolume.value(100) & hasWellDistRow.value(9) & hasWellDistCol.value(9) 
                                & hasDepthWell.value(14.5) & hasShapeWell.value("round") & hasShapeWellBottom.value("flat") 
                                & hasTopRadiusXY.value(4.5) & hasBottomRadiusXY.value(4.5) & hasBottomRadiusZ.value(0) 
                                & hasConeAngle.value(0) & hasConeDepth.value(0) & hasShapePolygonXY.value(0) 
                                & hasShapePolygonZ.value(0) & hasShapeModel2D.value("circle") & hasShapeModel3D.value("cylinder") 
                                & hasScrewCap.value(False) & hasScrewCapMaterial.value("N/A") & hasScrewCapColor.value("N/A") 
                                & hasColor.value("#FFFFFF") & hasMaterial.value("polystyrene") & hasMass.value(0) 
                                & hasMaxSheerForce.value(0) & hasCoatingMaterial.value("N/A") & hasSetptum.value(False) 
                                & hasSeptumMaterial.value("N/A") & hasSeptumPenetrationForce.value(0) & isLiddable.value(False) & isStackable.value(True) 
                                & isSealable.value(False) & hasManifacturer.value("N/A") & isProductType.value("N/A") & hasModelNumber.value("N/A") & hasProductNumber.value("N/A") ]
                
            

            


In [ ]:
lw = labop_Labware()
lw.define_ontology()

list(lw.lolw.classes())

## Defining individuals - Assertion Component (A-Box)

In [ ]:
with lw.lolw:
    greiner_384_v = lw.lolw.Labware("Greiner_384_V",
                                        hasLength=lw.lolw.Length(length=127.76),
                                        hasWidth=lw.lolw.Length(length=85.48),
                                        hasNumRows=16, 
                                        hasNumCols=24, 
                                        hasNumWells=384 )

    #plate_1536_well = lw.lolw.Labware("Plate1536Well", hasNumRows=32, hasNumCols=48, hasNumWells=1536)

    #plate_384_well = lw.lolw.Labware("Plate384Well", hasNumRows=16, hasNumCols=24, hasNumWells=384)

    #plate_96_well = lw.lolw.Labware("Plate96Well", hasNumRows=8, hasNumCols=12, hasNumWells=96)

    # slas_1_2004 = lw.lolw.Labware("SLAS_1_2004", hasNumRows=8, hasNumCols=12, hasNumWells=96) 

    # SLAS-2-2004-4

    # SLAS-2-2004-4-1

    # SLAS-2-2004-4-2

    # SLAS-4-2004


In [ ]:

greiner_384_v.is_a


In [ ]:
type(greiner_384_v)

In [ ]:
greiner_384_v.hasNumCols, greiner_384_v.hasNumWells

In [ ]:
hl = lw.lolw.hasLength

hl[greiner_384_v][0]



In [ ]:
greiner_384_v.hasLength


In [ ]:
greiner_384_v.hasLength.length

In [ ]:
greiner_384_v.hasLength.referenceUnit

In [ ]:
greiner_384_v.hasLength.referenceUnit.hasSymbolData

In [ ]:
greiner_384_v.hasWidth.length

In [ ]:
greiner_384_v.hasLength.referenceUnit

In [ ]:
greiner_384_v.hasWidth, greiner_384_v.hasWidth.referenceUnit, greiner_384_v.hasWidth.referenceUnit.hasSymbolData

In [ ]:
# loading an example labware catalog csv file:

print(os.getcwd())

strateos_csv = "./strateos_containers.csv"
strateos_cont_df = pd.read_csv(strateos_csv, delimiter=";")
strateos_cont_df = strateos_cont_df.reset_index()  # make sure indexes pair with number of rows
strateos_cont_df.head(32)

In [ ]:
strateos_cont_df[strateos_cont_df['Vendor'] == 'Greiner']

In [ ]:
with lw.lolw:
  for index,row in strateos_cont_df.iterrows():
    print(row['Id'], "-- >", row['Well Count'])
    law = lw.lolw.Labware( row['Id'],
                             hasManifacturer=row['Vendor'],
                             hasNumRows=row['Well Count'] / row['Column Count'], 
                             hasNumCols=row['Column Count'],
                             hasNumWells=row['Well Count'],
                             #hasHeight=row['Height (mm)'],
                             #hasWellVolume=row['Well Volume (ul)'],
                               )

## Competency questions to be answered by the ontology

In [ ]:
# provide width, length, height of a plate instance 

# in which unit is the width, length, height defined?


# which force is required to open the lid of a screw cap vial ?

# is a certain plate of type "SLAS1-2004compliant" ?

# Where can I punch the septum of a screw cap vial / GC- / HPLC vial ?

# What is the max. volume of a well of a multiwell plate ?

# at with height can a robot grab a microtiterplate with a lid ?



## SPARQL queries

In [ ]:
prefix_dict = {
    'rdf': "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    'rdfs': "http://www.w3.org/2000/01/rdf-schema#",
    'xml': "http://www.w3.org/XML/1998/namespace",
    'xsd': "http://www.w3.org/2001/XMLSchema#",
    'owl': "http://www.w3.org/2002/07/owl#",
    'skos': "http://www.w3.org/2004/02/skos/core#",
    'dc': "http://purl.org/dc/elements/1.1/",
    'dcterm': "http://purl.org/dc/terms/",
    'dctype': "http://purl.org/dc/dcmitype/",
    'foaf': "http://xmlns.com/foaf/0.1/",
    'wd': "http://www.wikidata.org/entity/",
    'ex': "http://www.example.com/",
    'emmo': "http://emmo.info/emmo#",
    'oso': "http://www.oso.org/oso#",
    'osom': "http://www.oso.org/oso/measurements#",
    'osolw': "http://www.oso.org/oso/labware#",
}

In [ ]:
graph = lw.emmo_world.as_rdflib_graph()

for prefix, iri in prefix_dict.items():
    print(prefix, "--- ", iri )
    graph.bind(prefix, iri)

In [ ]:
# get all labware from Greiner

query = """

PREFIX lolw: <http://www.labop.org/labware#>

SELECT ?lm 
WHERE {
    ?lm rdf:type lolw:Labware.
    ?lm lolw:hasManifacturer "Greiner".
    }
"""

In [ ]:
results = list(lw.emmo_world.sparql(query))
results

In [ ]:
# get all labware from Greiner with 384 wells 

query = """

PREFIX osolw: <http://www.oso.org/oso/labware#>

SELECT ?lm 
WHERE {
    ?lm rdf:type osolw:Labware.
    ?lm osolw:hasManifacturer "Greiner".
    ?lm osolw:hasNumWells 384.
    }
"""

In [ ]:
results = list(lw.emmo_world.sparql(query))
results

In [ ]:
results = list(lw.emmo_world.sparql(query))
results

## saving ontology as ttl file

### exporting ontology as ttl file

In [ ]:

# Save new ontology as owl
lw.lolw.sync_attributes(name_policy='uuid', class_docstring='elucidation',
                     name_prefix='labop_')
 
lw.lolw.set_version(version_iri=lw.labop_labware_version_iri)
lw.lolw.dir_label = False

lw.catalog_mappings[lw.labop_labware_version_iri] = lw.labop_labware_ttl_filename 

#################################################################
# Annotate the ontology metadata
#################################################################

lw.lolw.metadata.abstract.append(en(
        'An EMMO-based domain ontology forscientific labware.'
        'olw-measurement is released under the Creative Commons Attribution 4.0 '
        'International license (CC BY 4.0).'))


lw.lolw.metadata.title.append(en('LabOP-Labware'))
lw.lolw.metadata.creator.append(en('mark doerr'))
lw.lolw.metadata.contributor.append(en('university greifswald'))
lw.lolw.metadata.publisher.append(en(''))
lw.lolw.metadata.license.append(en(
    'https://creativecommons.org/licenses/by/4.0/legalcode'))
lw.lolw.metadata.versionInfo.append(en(lw.__version__))
lw.lolw.metadata.comment.append(en(
    'The EMMO requires FaCT++ reasoner plugin in order to visualize all'
    'inferences and class hierarchy (ctrl+R hotkey in Protege).'))
lw.lolw.metadata.comment.append(en(
    'This ontology is generated with data from the ASE Python package.'))
lw.lolw.metadata.comment.append(en(
    'Contacts:\n'
    'mark doerr\n'
    'University Greifswald\n'
    'email: mark.doerr@suni-greifswald.de\n'
    '\n'
    ))

lw.lolw.save(lw.labop_labware_ttl_filename , overwrite=True)
#olw.save(labop_measurement_owl_filename, overwrite=True)
write_catalog(lw.catalog_mappings)
# olw.sync_reasoner()
# olw.save('olw-measurement-inferred.ttl', overwrite=True)
# ...and to the sqlite3 database.
# world.save()


# Manually change url of EMMO to `emmo_url` when importing it to make
# it resolvable without consulting the catalog file.  This makes it possible
# to open the ontology from url in Protege
import rdflib  # noqa: E402, F401
g = rdflib.Graph()
g.parse(lw.labop_labware_ttl_filename , format='turtle')
for s, p, o in g.triples(
        (None, rdflib.URIRef('http://www.w3.org/2002/07/owl#imports'), None)):
    if 'emmo-inferred' in o:
        g.remove((s, p, o))
        g.add((s, p, rdflib.URIRef(lw.emmo_url)))
g.serialize(destination=lw.labop_labware_ttl_filename, format='turtle')

